# Imports

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import pymongo

# MongoDB Connection

In [ ]:
user = 'user'
password = 'pass'
node_ip = '192.168.2.20'
node_port = '30017'
url = f'mongodb://{user}:{password}@{node_ip}:{node_port}/'
client = pymongo.MongoClient(url, serverSelectionTimeoutMS = 2000)
client.server_info()

In [ ]:
print('Databases:', client.list_database_names())
db = client['test']
print('Collections:', db.list_collection_names())
collection = db['gpu-scrapper']
print('First Document:', str(collection.find_one())[:100])

# Get Data

In [ ]:
query = {
    'metadata.request_datetime_utc': {
        '$gte': dt.datetime.utcnow() - dt.timedelta(days=1)
    }
}

In [ ]:
everything = []
for x in collection.find(query):
    everything.append(x)
len(everything)

# Plot Data

In [ ]:
X = [x['metadata']['request_datetime_utc'] for x in everything if x['result'] is not None]

In [ ]:
Y_found = [x['result']['products_found'] for x in everything if x['result'] is not None]
Y_available = [x['result']['products_available'] for x in everything if x['result'] is not None]

In [ ]:
X_error = [x['metadata']['request_datetime_utc'] for x in everything if x['result'] is None]
Y_error = [-1 for x in everything if x['result'] is None]

In [ ]:
_, ax = plt.subplots(figsize=(16,4))
ax.scatter(X, Y_found)
ax.scatter(X, Y_available)
ax.scatter(X_error, Y_error, color='red')
ax.set_xlim((dt.datetime.utcnow() - dt.timedelta(days=1), dt.datetime.utcnow()))
plt.show()

# Check Error

In [ ]:
query = {
    'metadata.error_report': {
        '$exists': True
    }
}

In [ ]:
everything = []
for x in collection.find(query):
    everything.append(x)
len(everything)

In [ ]:
with open('page.html', 'w') as f:
    f.write(x['metadata']['listing_pages'][0])